In [86]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import geopandas as gpd
from shapely.geometry import Point, Polygon

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import precision_score, explained_variance_score

import zipfile
import requests
import os
import shutil

%matplotlib inline

In [77]:
#Get data
filestring = './data/anc_out.csv'
df = pd.read_csv(filestring)

df1 = df #copy it

df

,index_left,Util_Indx_BBL,countBBL,countIssued,month,pct_metro_coverage,NAME_ANC 1A,NAME_ANC 1B,NAME_ANC 1C,NAME_ANC 1D,...,NAME_ANC 7B,NAME_ANC 7C,NAME_ANC 7D,NAME_ANC 7E,NAME_ANC 7F,NAME_ANC 8A,NAME_ANC 8B,NAME_ANC 8C,NAME_ANC 8D,NAME_ANC 8E
0,0,0.054171,200.0,1.0,2012.01,0.001354,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0.030303,66.0,0.0,2012.01,0.050505,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,2,0.068413,235.0,3.0,2012.01,0.727116,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0.080778,748.0,7.0,2012.01,0.785084,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0.107343,421.0,1.0,2012.01,0.989271,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,5,0.025030,21.0,0.0,2012.01,0.000000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,6,0.028558,178.0,8.0,2012.01,0.197016,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,7,0.047672,301.0,3.0,2012.01,0.000000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,8,0.026827,116.0,4.0,2012.01,0.387604,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,9,0.026550,203.0,1.0,2012.01,0.074549,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [92]:
df = df1
df['y']= df['countIssued'].shift(-40)
df = df.dropna()
df = df.drop('index_left', axis= 1)
df = df.fillna(0)
np.any(np.isnan(df))

False

In [93]:
np.all(np.isfinite(df))

True

In [94]:
df.shape

(2360, 46)

In [95]:
data = df
data = data.reset_index()
cv = KFold(n_splits=10, shuffle=True, random_state=0)
for train_index, test_index in cv.split(data):
    ## Define training and test sets
    X_train = data.drop(['y'], axis=1)
    
    X_train = X_train.loc[train_index]
    y_train = data['y']
    y_train = y_train.loc[train_index]
    X_test  = data.loc[test_index]
    X_test  = X_test.drop(['y'], axis=1)
    y_test  = data['y']
    y_test  = y_test.loc[test_index]
    
       
    ## Fit model
    clf = RandomForestRegressor()
    clf.fit(X_train, y_train)

    ## Generate predictions
    predicted = clf.predict(X_test)
    
    
    ## Compare to actual outcomes and return precision
    print('Precision: '+str(100 * round(explained_variance_score
                                        (y_test, predicted))))

Precision: 100.0
Precision: 100.0
Precision: 100.0
Precision: 100.0
Precision: 100.0
Precision: 100.0
Precision: 100.0
Precision: 100.0
Precision: 100.0
Precision: 100.0
